# Langchain practise

In [1]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
import os


In [2]:
# load the apikey
_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [3]:
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY
)
